In [67]:
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [68]:
filter_source = ['All Source', 'Other Biomass', 'Other', 'Other Gases', 'Ohter Biomass', 'Wood and Wood Derived Fuels', 'Geothermal', 'Nuclear', 'Pumped Storage']
wanted_source = ['Coal', 'Petroleum', 'Hydroelectric', 'Natural Gas', 'Wind', 'Solar Thermal and Photovoltaic']
statelist = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 
             'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 
             'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [69]:
plants = pd.read_csv('plantcapacity.csv')
plants = plants.drop(['Producer Type','Generators','Nameplate Capacity (Megawatts)','Summer Capacity (Megawatts)'],axis = 1)
plants = plants[plants['Year'].isin([2015])] 

for item in filter_source:
    plants = plants[plants['Fuel Source'].str.contains(item) == False]

plants = plants.rename(columns = {'State Code':'State','Fuel Source':'Source'})    
plants.head()

,Year,State,Source,Facilities
37306,2015,AK,Coal,1
37308,2015,AK,Coal,2
37310,2015,AK,Petroleum,3
37312,2015,AK,Coal,2
37313,2015,AK,Hydroelectric,30


In [70]:
mod_plants = []
for state in statelist:
    state_plants = plants[plants['State'].isin([state])]
    state_plants['Facilities'] = state_plants['Facilities'].astype(int)
    state_plants = state_plants.groupby(['Year', 'State', 'Source'], as_index = False).agg(sum)
    mod_plants.append(state_plants)
mod_plants = pd.concat(mod_plants, axis=0).reset_index(drop = True)
mod_plants.head()

/Users/IvanC/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,State,Source,Facilities
0,2015,AK,Coal,10
1,2015,AK,Hydroelectric,60
2,2015,AK,Natural Gas,30
3,2015,AK,Petroleum,192
4,2015,AK,Wind,12


In [120]:
final_plants = pd.DataFrame()
final_plants['State'] = statelist
final_plants.insert(0, 'Year', 2015)

for item in wanted_source:
    source_df = mod_plants[mod_plants['Source'].isin([item])]
    source_df = source_df.drop(['Year', 'Source'], axis = 1)
    source_df = source_df.rename(columns = {'Facilities':item}) 
    final_plants = pd.merge(final_plants, source_df, on = 'State', how = 'left').fillna(0)
    final_plants[item] = final_plants[item].astype(int)
    

final_plants = final_plants.rename(columns = {'Petroleum':'Petro','Natural Gas':'NG','Hydroelectric':'Hydro','Solar Thermal and Photovoltaic':'Solar'})
final_plants = final_plants[['Year', 'State', 'Coal', 'NG', 'Petro', 'Hydro', 'Solar', 'Wind']]
final_plants

,Year,State,Coal,NG,Petro,Hydro,Solar,Wind
0,2015,AK,10,30,192,60,0,12
1,2015,AL,14,54,8,48,0,0
2,2015,AR,10,36,6,38,2,0
3,2015,AZ,12,62,8,20,118,10
4,2015,CA,4,580,30,502,742,230
5,2015,CO,26,62,26,84,58,48
6,2015,CT,2,68,56,28,6,0
7,2015,DC,0,2,0,0,0,0
8,2015,DE,2,20,10,0,14,2
9,2015,FL,26,142,42,4,26,0


In [121]:
final_plants.to_csv('final_plants.csv')